In [130]:
import censusdata
import pandas as pd

#function to make list of all county ids in state (given by census state id)
def county_list(state_number):
    counties = censusdata.geographies(censusdata.censusgeo([('state', state_number),('county','*')]), 'acs5', 2018)
    county_list = []
    for i in counties.keys():
        county_list.append(counties[i].geo[1][1])
    return county_list

#function to pull defined variables for blocks in specified state, looping over counties (can't pull all blocks in a state)
#(input state id and list of variables)
def block_pull(state_id,variable_list):
    c_list = county_list(state_id)
    for i in range(0,len(c_list)):
        geo = censusdata.censusgeo([('state',state_id),('county',c_list[i]),('tract','*'),('block group','*')])
        county_df = censusdata.download('acs5',2018,geo,variable_list)
        if i == 0:
            data = county_df
        else:
            data = pd.concat([data,county_df])
    return data

#just one variable in here for now, need to select and add variables
variables_list = ['B02001_001E']

block_data = block_pull('39',variables_list)

In [131]:
print(block_data)

                                                    B02001_001E
Block Group 3, Census Tract 9540, Gallia County...         1348
Block Group 2, Census Tract 9540, Gallia County...          621
Block Group 1, Census Tract 9540, Gallia County...         1125
Block Group 2, Census Tract 9539, Gallia County...         1640
Block Group 1, Census Tract 9539, Gallia County...          906
Block Group 4, Census Tract 9539, Gallia County...         1360
Block Group 2, Census Tract 9535, Gallia County...          962
Block Group 3, Census Tract 9535, Gallia County...          718
Block Group 1, Census Tract 9535, Gallia County...          621
Block Group 1, Census Tract 9536, Gallia County...         1043
Block Group 2, Census Tract 9536, Gallia County...         1720
Block Group 3, Census Tract 9536, Gallia County...         1450
Block Group 2, Census Tract 9537, Gallia County...         1682
Block Group 1, Census Tract 9537, Gallia County...         1256
Block Group 3, Census Tract 9537, Gallia